In [199]:
import pandas as pd
import numpy as np
import seaborn as sns

In [175]:
an13 = pd.read_csv("an13.csv", na_values = "NA", encoding = "latin1")

an13.drop(["Unnamed: 0", "professione"], axis = 1, inplace = True)
an13.drop(["sconto", "riduzione","agenzia","cap"], axis = 1, inplace = True)

# "nuovo_abb" is alrady dichotomous and with full values. Just convert it into 0/1
an13["nuovo_abb"] = [1 if x == "NUOVO ABBONATO" else 0 for x in an13["nuovo_abb"]]

# "sesso" has missing values. We filled the according to the distribution of non-NaN values
count_sex = an13["sesso"].value_counts(normalize=True)
missing_sex = an13["sesso"].isnull()
an13.loc[an13["sesso"].isnull(),"sesso"] = np.random.choice(count_sex.index,
                                                            size = len(an13[missing_sex]),
                                                            p = count_sex.values)
an13.head()

,codcliente,data_inizio,importo,tipo_pag,agenzia_tipo,sesso,data_nascita,comune,nuovo_abb
0,90095,09/12/2012 00:00,28,CONTANTI,PUNTO INFORMATIVO,M,1985,TORINO,1
1,89662,06/01/2013 00:00,28,CONTANTI,PUNTO INFORMATIVO,M,1985,TORINO,1
2,202278,27/01/2013 00:00,28,BANCOMAT,PUNTO INFORMATIVO,M,1992,TORINO,1
3,269895,02/12/2012 00:00,28,CONTANTI,PUNTO COMMERCIALE,M,1986,SVIZZERA,1
4,88177,26/01/2013 00:00,28,CONTANTI,PUNTO INFORMATIVO,M,1983,TORINO,1


In [178]:
in13 = pd.read_csv("in13.csv", na_values = "NA")
in13.drop("Unnamed: 0", axis = 1, inplace = True)
in13.rename({"CodCliente": "codcliente"}, axis = 1, inplace = True)

un_mus = pd.read_excel("unallocated_museums.xlsx")
in13 = in13.merge(un_mus, how = "left", on = "museo")
in13.drop(["comune.museo","prov.museo"], axis = 1, inplace = True)
in13.head()

,datai,orai,importo,museo,prov_museo,com_museo,codcliente,tipo.museo
0,01/12/2012,10:10,7.5,REGGIA DI VENARIA REALE,TO,VENARIA REALE,46715,Storia
1,01/12/2012,10:11,5.0,REGGIA DI VENARIA REALE,TO,VENARIA REALE,46715,Storia
2,01/12/2012,10:11,7.5,REGGIA DI VENARIA REALE,TO,VENARIA REALE,133015,Storia
3,01/12/2012,10:12,5.0,REGGIA DI VENARIA REALE,TO,VENARIA REALE,133015,Storia
4,01/12/2012,10:18,2.5,MUSEO DELLA FRUTTA,TO,TORINO,80732,Arte


In [230]:
in13["tipo.museo"].value_counts()

Arte                    251638
Storia                  195596
Castello                 55274
Religione                16495
Parco                    10142
Sport                     5375
Storia/ Architettura      5077
Scienza                   4902
Bambini                    322
Scienze naturali           168
Archeologia                 77
Chiesa                      19
Name: tipo.museo, dtype: int64

In [202]:
a = pd.to_datetime(in13["datai"], format = "%d/%m/%Y")
b = pd.to_datetime(in13["orai"], format = "%H:%M")

In [218]:
for x in a:
    print(  )

In [227]:
# From "in13", we've built a new dataframe with the following columns:
# - codcliente
# - frequenza: number of times every customer went to museums
# - variety: how many different museums they have visited
# - passione: given different museum's categories, this score measures how many times a customer
#             prefers a category. If they did only one visit, the score is set just below the 0.8 thershold
# - appassionato: takes the "passione" scores into a dichotomous variable at a 0.8 threshold.
#                 The result is that only 20% of the customer is defined as passionate, which may be closed to reality

in13_custom = pd.DataFrame({"codcliente":pd.DataFrame(in13["codcliente"].value_counts()).index,
                            "frequenza":pd.DataFrame(in13["codcliente"].value_counts())["codcliente"]})
                            
in13_custom["variety"] = [ len( in13[in13["codcliente"] == x]["museo"].unique() )
                          for x in in13_custom["codcliente"] ]

in13_custom["passione"] = [ (in13[in13["codcliente"] == x]["tipo.museo"].value_counts()[0])/(in13_custom[in13_custom["codcliente"] == x]["frequenza"].values[0] )
                           for x in in13_custom["codcliente"] ]

in13_custom["passione"] = [in13_custom.loc[x]["passione"] if in13_custom.loc[x]["frequenza"] > 1 else 0.79
                           for x in in13_custom["codcliente"] ]

in13_custom["appassionato"] = [1 if x >= 0.8 else 0 for x in in13_custom["passione"] ]

in13_custom

,codcliente,frequenza,variety,passione,appassionato
219314,219314,264,66,0.477273,0
48954,48954,179,79,0.329609,0
185108,185108,124,30,0.370968,0
172643,172643,101,20,0.534653,0
132550,132550,94,24,0.606383,0
...,...,...,...,...,...
109849,109849,1,1,0.790000,0
258105,258105,1,1,0.790000,0
233525,233525,1,1,0.790000,0
91392,91392,1,1,0.790000,0


In [220]:
data1 = pd.read_csv("data1.csv")

# "abb14" and "ultimo_ing.x" are droped due to their NaN values
data1.drop(["Unnamed: 0", "abb14", "ultimo_ing.x"], axis = 1, inplace = True)
data1.head()

,codcliente,si2014,abb13
0,100005,0,2013-01-27
1,100006,1,2013-01-14
2,100007,0,2012-12-05
3,100017,1,2012-12-10
4,100020,0,2012-12-09
